# Домашнее задание Парсинг данных
# Pro

Возьмите базу квартир (moscow.csv), проведите парсинг и очистку данных:

● Уберите строки “Для заметок”

● Уберите столбцы “ГРМ” и “Примечания”

● Для столбца “Балкон” добавьте категорию “Отсутствует” (вместо NaN)

● Значения столбца “Бонус агенту” переведите в тысячи рублей, а пустые значения
заполните нулями

● В столбец “Площадь” запишите общую площадь всех помещений (формат записи данных в столбце "64.1/23/20", где 64.1 - общая площадь всех помещений)

● Для столбца “Санузел” заполните отсутствующие значения строкой “Нет данных”

● Столбец “Цена” переведите в тысячи рублей

**ПРИМЕЧАНИЕ:** Чтобы правильно загрузить датасет moscow.csv, нужно указать параметр `sep = ;` в функции `pd.read_csv()`

In [ ]:
# Статический вывод графики (графики отображаются в той ячейке, в которой используется plt.show())
import pandas as pd # Импортируем библиотеку обработки и анализа данных pandas
import matplotlib.pyplot as plt # Импортируем модуль pyplot библиотеки matplotlib для построения графиков
import datetime                 # Импортируем модуль datetime для обработки данных в формете даты
import numpy as np              # Импортируем библиотеку numpy
from google.colab import files  # Импортируем библиотеку files для работы с файлами
import re                       # Регулярные выражения

# Рисовать графики сразу же
%matplotlib inline

In [ ]:
# Для загрузки данных используем drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/My Drive/БД/moscow.csv', sep = ';')

In [ ]:
df.head(5)

,Комнат,Метро / ЖД станции,От станции,Дом,Балкон,Санузел,Площадь,"Цена, руб.",ГРМ,Бонус агенту,Дата,Кол-во дней в экспозиции,Источник,Примечание
0,1,Шелепиха м.,12п,35/37 М,NaN,2,64.1/23/20,19500000.0,NaN,NaN,25.07.2019,1.0,Привлекательные,Собственность оформлена! Ключи на руках! Опера...
1,Для заметок:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Пятницкое шоссе м.,30т,11/14 П,Л,Р,38.4/17/9,4800000.0,NaN,NaN,25.07.2019,2.0,Привлекательные,Зеленоград к.1820 однокомнатная квартира общ.3...
3,Для заметок:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Планерная м.,5т,11/22 П,Б,С,38.2/19.9/9.9,7990000.0,NaN,NaN,25.07.2019,4.0,Привлекательные,"Уютная, светлая квартира, застекленный балкон ..."


In [ ]:
df_ = df[::2]
df_ = df_.reset_index(drop = True)
df_ = df_.drop(['ГРМ', 'Примечание'], axis = 1)
df_['Балкон'] = df_['Балкон'].fillna('Отсутствует')
df_['Санузел'] = df_['Санузел'].fillna('Нет данных')
df_['Бонус агенту'] = df_['Бонус агенту'].fillna(0)

In [ ]:
bonus = df_['Бонус агенту']
cost = df_['Цена, руб.']
for i in range(bonus.shape[0]):

  if isinstance(bonus[i], str) and bonus[i].find('руб.') != -1:
    bonus[i] = bonus[i][:bonus[i].find('руб.')]
    bonus[i] = bonus[i].replace(' ', '')
    bonus[i] = float(bonus[i]) / 1000

  elif isinstance(bonus[i], str) and bonus[i].find('%') != -1:
    bonus[i] = bonus[i][:bonus[i].find('%')]
    bonus[i] = bonus[i].replace(' ', '')
    bonus[i] = float(bonus[i]) / 100
    bonus[i] = bonus[i] * cost[i]
    bonus[i] = bonus[i] / 1000

df_['Бонус агенту'] = bonus

In [ ]:
square = df_['Площадь']
arr = []
for i in range(square.shape[0]):
  el = square[i]
  ind = el.find('/')
  el = el[:ind]
  arr.append(el)
df_['Площадь'] = arr

In [ ]:
for i in range(cost.shape[0]):
  cost[i] = cost[i] / 1000
df_['Цена, руб.'] = cost


In [ ]:
df_.rename(columns = {'Цена, руб.':'Цена, тыс. руб.', 'Бонус агенту':'Бонус агенту, тыс. руб.'})

,Комнат,Метро / ЖД станции,От станции,Дом,Балкон,Санузел,Площадь,"Цена, тыс. руб.","Бонус агенту, тыс. руб.",Дата,Кол-во дней в экспозиции,Источник
0,1,Шелепиха м.,12п,35/37 М,Отсутствует,2,64.1,19500.00,0,25.07.2019,1.0,Привлекательные
1,1,Пятницкое шоссе м.,30т,11/14 П,Л,Р,38.4,4800.00,0,25.07.2019,2.0,Привлекательные
2,1,Планерная м.,5т,11/22 П,Б,С,38.2,7990.00,0,25.07.2019,4.0,Привлекательные
3,1,Шаболовская м.,12п,8/12 ?,Б,Р,38,11200.00,0,25.07.2019,7.0,Привлекательные
4,1,Бабушкинская м.,15п,4/14 П,Отсутствует,С,34,6250.00,0,25.07.2019,11.0,Привлекательные
...,...,...,...,...,...,...,...,...,...,...,...,...
126263,Ст,Некрасовка м.,4п,2/17 П,Отсутствует,Нет данных,19.8,2850.00,0,12.07.2019,14.0,AVITO.ru
126264,Ст,Авиамоторная м.,6п,2/8 К,Отсутствует,С,20.4,3839.99,0,23.07.2019,14.0,Яндекс
126265,Ст,Коломенская м.,15п,1/9 П,Отсутствует,Нет данных,15.4,2900.00,0,12.07.2019,14.0,AVITO.ru
126266,Ст,Алтуфьево м.,5т,4/9 М,Отсутствует,Нет данных,26,5200.00,0,22.07.2019,14.0,AVITO.ru
